In [2]:
import subprocess

packages = [
    "transformers",
    "torch",
    "pandas",
    "numpy"
]

for package in packages:
    subprocess.run(["pip", "install", package])


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 115.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.1/774.1 kB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 54.7 MB/s eta 0:00:00a 0:00:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 130.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 52.6 MB/s eta 0:00:00


In [3]:
#from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
import numpy as np
from transformers import BertModel, BertTokenizer

In [4]:
data = pd.read_csv('amazon_search_hedonic_clean.csv', sep = ";")

In [5]:
data.describe()

,rating,helpful,n_pictures,review_len,days_diff
count,114978.000000,114978.000000,114978.000000,114978.000000,114978.000000
mean,4.104316,1.237689,0.167102,314.651672,659.717416
std,1.297240,2.128697,0.746349,409.178698,962.037709
min,1.000000,0.000000,0.000000,1.000000,0.000000
25%,4.000000,0.000000,0.000000,81.000000,80.000000
50%,5.000000,0.000000,0.000000,185.000000,253.000000
75%,5.000000,2.000000,0.000000,387.000000,821.000000
max,5.000000,11.000000,25.000000,9198.000000,8060.000000


In [26]:
data.isna().any()

asin_url         False
link             False
product_title    False
name_x            True
rating           False
review_title     False
country          False
date             False
specs            False
review           False
helpful          False
picture          False
verified         False
profile           True
video             True
scrape_date      False
profile_url      False
asin             False
category         False
n_pictures       False
review_len       False
days_diff        False
review_clean     False
dtype: bool

In [6]:
data['review'] = data['review'].fillna('')

In [7]:
data['id'] = data.index

In [53]:
len(data)

114978

In [8]:
# Load the pre-trained BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [26]:
import torch

# Define a function to extract features for each transaction
def extract_features(text, max_length, model, tokenizer, device):
    # Truncate text if it exceeds the maximum length
    if len(text) > max_length:
        text = text[:max_length]
    # Tokenize the text
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)]).to(device)
    # Get the hidden states for each token
    with torch.no_grad():
        outputs = model(input_ids)
        hidden_states = outputs[2]
    # Concatenate the last 4 hidden states
    token_vecs = []
    for layer in range(-4, 0):
        token_vecs.append(hidden_states[layer][0])
    # Calculate the mean of the last 4 hidden states
    features = []
    for token in token_vecs:
        features.append(torch.mean(token, dim=0))
    # Return the features as a tensor
    return torch.stack(features)

# Example usage
text = "Example text to extract features from."
max_length = 128

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Call the function with the model, tokenizer, and device
features = extract_features(text, max_length, model, tokenizer, device)

In [27]:
features

tensor([[-0.2141,  0.0289,  0.0764,  ..., -0.2412, -0.3530,  0.3690],
        [-0.3596, -0.2697,  0.0532,  ..., -0.3237, -0.5377,  0.2473],
        [-0.2519, -0.2501, -0.0483,  ..., -0.5202, -0.4691,  0.2371],
        [-0.0690, -0.3024,  0.0601,  ..., -0.1918, -0.2589,  0.1523]],
       device='cuda:0')

In [30]:
# Extract features for each transaction
features = []
max_length = 128
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for i in range(len(data)):
    if i % 100 == 0:
      print(i)
    #features.append(extract_features(data.iloc[i]["review"], max_length = 128))
    features.append(extract_features(text, max_length, model, tokenizer, device))
# Concatenate the features and convert to a numpy array
#features = torch.cat(features).numpy()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [40]:
features

[tensor([[-0.2141,  0.0289,  0.0764,  ..., -0.2412, -0.3530,  0.3690],
         [-0.3596, -0.2697,  0.0532,  ..., -0.3237, -0.5377,  0.2473],
         [-0.2519, -0.2501, -0.0483,  ..., -0.5202, -0.4691,  0.2371],
         [-0.0690, -0.3024,  0.0601,  ..., -0.1918, -0.2589,  0.1523]],
        device='cuda:0'),
 tensor([[-0.2141,  0.0289,  0.0764,  ..., -0.2412, -0.3530,  0.3690],
         [-0.3596, -0.2697,  0.0532,  ..., -0.3237, -0.5377,  0.2473],
         [-0.2519, -0.2501, -0.0483,  ..., -0.5202, -0.4691,  0.2371],
         [-0.0690, -0.3024,  0.0601,  ..., -0.1918, -0.2589,  0.1523]],
        device='cuda:0'),
 tensor([[-0.2141,  0.0289,  0.0764,  ..., -0.2412, -0.3530,  0.3690],
         [-0.3596, -0.2697,  0.0532,  ..., -0.3237, -0.5377,  0.2473],
         [-0.2519, -0.2501, -0.0483,  ..., -0.5202, -0.4691,  0.2371],
         [-0.0690, -0.3024,  0.0601,  ..., -0.1918, -0.2589,  0.1523]],
        device='cuda:0'),
 tensor([[-0.2141,  0.0289,  0.0764,  ..., -0.2412, -0.3530,  0.369

In [42]:
numpy_arrays = [tensor.cpu().numpy() for tensor in features]

In [51]:
numpy_arrays = np.array(numpy_arrays).reshape(len(data), -1)

In [54]:
features_df = pd.DataFrame(numpy_arrays)

In [55]:
features_df.to_csv("features_128.csv", sep = ";", header = False, index = True)

In [10]:
features_reshaped = features.reshape((1000, -1))
labels = data['helpful'].values
# concatenate the feature array with the label array horizontally
dataset = np.hstack((features_reshaped, labels.reshape((-1, 1))))

In [56]:
#regression

In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Convert the training and testing sets back into separate feature and label arrays
X_train, y_train = train_data[:, :-1], train_data[:, -1]
X_test, y_test = test_data[:, :-1], test_data[:, -1]

In [ ]:
#REGRESSION

In [ ]:
from sklearn.preprocessing import StandardScaler

# Assume you have an array 'data'
scaler = StandardScaler()
y_train_n = scaler.fit_transform(y_train.reshape(-1, 1))
y_test_n = scaler.transform(y_test.reshape(-1,1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Define the model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Input layer
#model.add(Dropout(0.25))
#model.add(Dense(128, activation = 'relu'))
#model.add(Dropout(0.25))
model.add(Dense(64, activation = 'relu'))
#model.add(Dropout(0.25))
model.add(Dense(1))  # Output layer

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train_n, epochs=50, batch_size=32, validation_data=(X_test, y_test_n) )

Epoch 1/50
25/25 [==============================] - 3s 16ms/step - loss: 1.2873 - val_loss: 1.1187
Epoch 2/50
25/25 [==============================] - 0s 6ms/step - loss: 0.8457 - val_loss: 1.0573
Epoch 3/50
25/25 [==============================] - 0s 5ms/step - loss: 0.8321 - val_loss: 1.1804
Epoch 4/50
25/25 [==============================] - 0s 5ms/step - loss: 0.7560 - val_loss: 1.0178
Epoch 5/50
25/25 [==============================] - 0s 5ms/step - loss: 0.7048 - val_loss: 1.0411
Epoch 6/50
25/25 [==============================] - 0s 8ms/step - loss: 0.6492 - val_loss: 1.1251
Epoch 7/50
25/25 [==============================] - 0s 5ms/step - loss: 0.5691 - val_loss: 1.1149
Epoch 8/50
25/25 [==============================] - 0s 5ms/step - loss: 0.5205 - val_loss: 1.2360
Epoch 9/50
25/25 [==============================] - 0s 5ms/step - loss: 0.4868 - val_loss: 1.1176
Epoch 10/50
25/25 [==============================] - 0s 6ms/step - loss: 0.3831 - val_loss: 1.2144
Epoch 11/50
25/25 

In [ ]:
y_pred = model.predict(X_test)

7/7 [==============================] - 0s 4ms/step


In [ ]:
np.mean((y_pred - y_test)**2)

6.96880900720273

In [ ]:
#CLASSIFICATION

In [ ]:
y_train_d = np.array([0 if i == 0 else 1 for i in y_train])
y_test_d = np.array([0 if i == 0 else 1 for i in y_test])

In [ ]:
# Define the model
from keras.regularizers import l2

model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.1)))  # Input layer
model.add(Dropout(0.25))
model.add(Dense(64, activation = 'relu', kernel_regularizer=l2(0.1)))
model.add(Dropout(0.25))
#model.add(Dense(64, activation = 'relu'))
#model.add(Dropout(0.25))
model.add(Dense(1, activation = 'sigmoid'))  # Output layer

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_d, epochs=50, batch_size=32, validation_data=(X_test, y_test_d) )

Epoch 1/50
25/25 [==============================] - 2s 24ms/step - loss: 19.5870 - accuracy: 0.5938 - val_loss: 9.4361 - val_accuracy: 0.6950
Epoch 2/50
25/25 [==============================] - 0s 8ms/step - loss: 6.3307 - accuracy: 0.7063 - val_loss: 4.3915 - val_accuracy: 0.6850
Epoch 3/50
25/25 [==============================] - 0s 10ms/step - loss: 3.5454 - accuracy: 0.7287 - val_loss: 2.8913 - val_accuracy: 0.6100
Epoch 4/50
25/25 [==============================] - 0s 12ms/step - loss: 2.4106 - accuracy: 0.7038 - val_loss: 2.0509 - val_accuracy: 0.5650
Epoch 5/50
25/25 [==============================] - 0s 9ms/step - loss: 1.6958 - accuracy: 0.6975 - val_loss: 1.4579 - val_accuracy: 0.6250
Epoch 6/50
25/25 [==============================] - 0s 9ms/step - loss: 1.2411 - accuracy: 0.7025 - val_loss: 1.1351 - val_accuracy: 0.6050
Epoch 7/50
25/25 [==============================] - 0s 9ms/step - loss: 0.9972 - accuracy: 0.6988 - val_loss: 0.9764 - val_accuracy: 0.5700
Epoch 8/50
25/25

In [ ]:
#IMAGE ANALYSIS

In [ ]:
!cp -r "/content/drive/My Drive/Thesis DSS/photos" "/content/"


In [ ]:
!ls "/content/"

drive  photos  sample_data


In [ ]:
import os
photos = os.listdir("/content/photos")

In [ ]:
photos

['883d7ec5-e5c4-47d9-b9e0-02e8b920f674._CR0,0,336,336_SX48_.jpg',
 '26e7fdd2-9e12-41f9-9426-13801141fa76._CR0,0,500,500_SX48_.jpg',
 '845389fa-6cec-4ea5-a101-2ebaaa429dcf._CR62,0,375,375_SX48_.jpg',
 '5aebb2fb-3541-40e0-854f-cd328a57c52d._CR0,0,500,500_SX48_.jpg',
 '6807898e-7bbc-4789-862f-69834940855b._CR0,0,500,500_SX48_.jpg',
 '1daa0e6a-5239-42d7-bfc8-58342cd18fb6._CR0,76.0,181,181_SX48_.jpg',
 '6468fa97-1b34-49bf-85af-7955aee07e24._CR0,0,375,375_SX48_.jpg',
 '31befOkEC6L._CR0,0,160,160_SX48_.gif',
 '4987a800-a35d-4306-908b-77fe481158bb._CR0,0,430,430_SX48_.jpg',
 '4fc25eb7-75e5-43a7-8bff-1a83ec7b4c0b._CR0,0,1199,1199_SX48_.jpg',
 '5182a520-e84c-48d3-bfec-af1cbbcfd1b6._CR0,26.0,281,281_SX48_.jpg',
 '31Up11VgPcL._CR0,0,100,100_UX110_.gif',
 '3f57238e-de09-4c69-b2ed-4d399ace87b0._CR50,0,300,300_SX48_.jpg',
 '8812995c-a0a9-49ac-9531-7365a755cf5a._CR0,0,1125,1125_SX48_.jpg',
 '069411ca-2d46-465c-b16a-fbdfc200b5cd._CR0,0,409,409_SX48_.jpg',
 '50f32ab3-5ef1-469c-a342-1c9106f605ab._CR83,0,

In [ ]:
from PIL import Image
image_data = []
labels = []
for p in photos:
  if p[-4:] != '.gif':
    image = Image.open("/content/photos/" + p)
    image_data.append(image)
    labels.append(p)
  else:
    # Handle the error
    print("Gif: " + p)

Gif: 31befOkEC6L._CR0,0,160,160_SX48_.gif
Gif: 31Up11VgPcL._CR0,0,100,100_UX110_.gif
Gif: 21-bC3PGHFL._CR1,0,102,102_UX110_.gif
Gif: 21n6d8qIWNL._CR0,0,432,432_SX48_.gif
Gif: 21anBGNomyL._CR0,0,100,100_UX110_.gif
Gif: 41QOCG7wlXL._CR25,0,210,210_SX48_.gif
Gif: 11o58WAFMhL._CR0,0,100,100_UX110_.gif
Gif: 31KS2A+wHJL._CR2,0,281,281_SX48_.gif
Gif: 71AYd8AR9ML._CR0,0,359,359_SX48_.gif
Gif: 41eshFVYrYL._CR0,0,400,400_SX48_.gif
Gif: 31tyBC8a7DL._CR0,0,175,175_SX48_.gif
Gif: 41eeWnTfpML._CR0,0,164,164_SX48_.gif
Gif: 310haZrArnL._CR40,0,240,240_SX48_.gif
Gif: 31ZhRMqn7BL._CR0,0,120,120_SX48_.gif
Gif: 51-eKR8WLnL._CR60,0,239,239_SX48_.gif
Gif: 11+zxWPHLLL._CR6,0,60,60_UX110_.gif
Gif: 71mXZ1A0mML._CR65,0,370,370_SX48_.gif


In [ ]:
image_matrix = np.array([np.array(img) for img in image_data[0:94]])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (94, 48, 48) + inhomogeneous part.

In [ ]:
photos[11][-4:] == ".gif"

True

In [ ]:
photos[93]

'106496a9-1311-4ce8-904d-0d21cefd7b3c._CR0,46.0,367,367_SX48_.jpg'

In [ ]:
np.array(image_data[93]).shape

(48, 48)

In [ ]:
data['profile'] = data['profile'].fillna('')

In [ ]:
def return_link(url):
  return url.split('/')[-1]
data['profile'] = data['profile'].apply(return_link)

In [ ]:
df_photos = pd.DataFrame({'photo_link' : photos[0:10]})
df_photos = pd.merge(df_photos, data, how = 'left', left_on= 'photo_link', right_on = 'profile')
df_photos['helpful'] = df_photos['helpful'].fillna(0)
df_photos = df_photos.drop_duplicates(subset = ['photo_link'])

In [ ]:
y_train = df_photos['helpful']

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dense
from keras.optimizers import Adam, RMSprop, SGD, Adagrad, Adadelta, Nadam, Adamax, AdamW

In [ ]:
from tensorflow.keras.applications import ResNet50
base_model = ResNet50(weights='imagenet', include_top=False)
#freeze layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model = models.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    layers.Dense(256, activation = 'swish'),
    BatchNormalization(),
    #layers.Dense(256, activation = 'relu'),
    layers.Dense(128, activation = 'swish'),
    BatchNormalization(),
    layers.Dense(32, activation = 'swish'),
    Dense(1)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(np.array(image_matrix), np.array(y_train).astype('float32'), epochs=4)

# Print model summary
model.summary()

Epoch 1/4
1/1 [==============================] - 10s 10s/step - loss: 431.3116
Epoch 2/4
1/1 [==============================] - 0s 37ms/step - loss: 351.8224
Epoch 3/4
1/1 [==============================] - 0s 38ms/step - loss: 315.1016
Epoch 4/4
1/1 [==============================] - 0s 33ms/step - loss: 289.1934
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, None, None, 2048   23587712  
                             )                                   
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
 0 (GlobalAveragePooling2D)                                      
                                                                 
 dense_40 (Dense)            (None, 256)               524544    
                                                                 
 